In [1]:
import string

import numpy as np
import scipy
import pandas as pd
import re
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

import gensim
from gensim.corpora import Dictionary
from gensim import models
from gensim.models import LdaModel
from gensim.matutils import corpus2csc

import html

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

C:\Users\Enno\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_json('venv/data/presidential_speeches.json') # load data
data.sort_values(by=['date'], inplace=True) # sort by data
data.reset_index(drop=True, inplace=True) # reset index

In [3]:
# sort presidents to their parties
president_list = []
for i in range(len(data['president'].unique())):
    president_list.append(data['president'].unique()[i])
parties_list= ['Federalist', 'Democratic-Republican', 'National Republican', 'Democratic', 'Whig', 'Republican', 'Democratic (Union)']
sort_party = [0, 0, 1, 1, 1, 2, 3, 3, 4, 4, 3, 4, 4, 5, 3, 3, 6, 5, 5, 5, 5, 3, 5, 5, 5, 5, 3, 5, 5, 5, 3, 3, 5, 5, 3, 3, 5, 5, 3, 5, 3, 5, 3, 5, 3]
party = []
for i in range(len(sort_party)):
    party.append(parties_list[sort_party[i]])
data['party'] = pd.Series(dtype='string')
for i in range(len(president_list)):
    data['party'][data['president'] == president_list[i]] = party[i]
# assign each speech its respective era
data['era'] = pd.Series(dtype='string')
era_list = ['Early Republic', 'Jacksonian Democracy', 'Sectional Conflict', 'Gilded Age', 'Progressive Era', 'Depression and World Conflict', 'Social Change and Soviet Relations', 'Globalization']
for i in range(len(data)):
    if data['date'][i] < pd.Timestamp('1829-01-01T12'):
        data['era'][i] =  era_list[0]
    if pd.Timestamp('1829-01-01T12') <= data['date'][i] < pd.Timestamp('1853-01-01T12'):
        data['era'][i] = era_list[1]
    elif pd.Timestamp('1853-01-01T12') <= data['date'][i] < pd.Timestamp('1881-01-01T12'):
        data['era'][i] = era_list[2]
    elif pd.Timestamp('1881-01-01T12') <= data['date'][i] < pd.Timestamp('1897-01-01T12'):
        data['era'][i] = era_list[3]
    elif pd.Timestamp('1897-01-01T12') <= data['date'][i] < pd.Timestamp('1921-01-01T12'):
        data['era'][i] = era_list[4]
    elif pd.Timestamp('1921-01-01T12') <= data['date'][i] < pd.Timestamp('1961-01-01T12'):
        data['era'][i] = era_list[5]
    elif pd.Timestamp('1961-01-01T12') <= data['date'][i] < pd.Timestamp('1989-01-01T12'):
        data['era'][i] = era_list[6]
    elif pd.Timestamp('1989-01-01T12') <= data['date'][i]:
        data['era'][i] = era_list[7]

In [4]:
data['transcript'].replace(to_replace='(Applause.)', regex=True, value='', inplace=True)
data['transcript'].replace(to_replace='(Laughter.)', regex=True, value='', inplace=True)

In [5]:
for i in range(len(data)):
    data['transcript'][i] = data['transcript'][i].replace('\'', '')

C:\Users\Enno\AppData\Local\Temp/ipykernel_2632/1438919703.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['transcript'][i] = data['transcript'][i].replace('\'', '')


In [6]:
import string
import re
import contractions

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from bs4 import BeautifulSoup
from textblob import TextBlob
from unidecode import unidecode

In [7]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Enno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Enno\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Enno\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Enno\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Enno\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
def lemmatize_pos_tagged_text(text, lemmatizer, pos_tag_dict):
  sentences = nltk.sent_tokenize(text)
  new_sentences = []

  for sentence in sentences:
    sentence = sentence.lower()
    new_sentence_words = []
    #one pos_tuple for sentence
    pos_tuples = nltk.pos_tag(nltk.word_tokenize(sentence)) 

    for word_idx, word in enumerate(nltk.word_tokenize(sentence)):
      nltk_word_pos = pos_tuples[word_idx][1]
      wordnet_word_pos = pos_tag_dict.get(
                          nltk_word_pos[0].upper(), None)
      if wordnet_word_pos is not None:
        new_word = lemmatizer.lemmatize(word, wordnet_word_pos)
      else:
        new_word = lemmatizer.lemmatize(word)

      new_sentence_words.append(new_word)

    new_sentence = " ".join(new_sentence_words)
    new_sentences.append(new_sentence)

  return " ".join(new_sentences)

In [9]:

class NltkPreprocessingSteps:
  def __init__(self, X):
    self.X = X

    self.sw_nltk = stopwords.words('english')
    new_stopwords = ['<*>', 'q', 'yeah', 'guy', 'okay', 'ahead', 'mr', 'st']
    self.sw_nltk.extend(new_stopwords)
    self.sw_nltk.remove('not')

    self.pos_tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}

    # '!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~' 32 punctuations in python
    # we dont want to replace . first time around
    self.remove_punctuations = string.punctuation.replace('.','')

  def remove_html_tags(self):
    self.X = self.X.apply(
            lambda x: BeautifulSoup(x, 'html.parser').get_text())
    return self

  def replace_diacritics(self):
    self.X = self.X.apply(
            lambda x: unidecode(x, errors="preserve"))
    return self

  def to_lower(self):
    self.X = np.apply_along_axis(lambda x: x.lower(), self.X)
    return self

  def expand_contractions(self):
    self.X = self.X.apply(
            lambda x: " ".join([contractions.fix(expanded_word) 
                        for expanded_word in x.split()]))
    return self

  def remove_numbers(self):
    self.X = self.X.apply(lambda x: re.sub(r'\d+', '', x))
    return self

  def replace_dots_with_spaces(self):
    self.X = self.X.apply(lambda x: re.sub("[.]", " ", x))
    return self

  def remove_punctuations_except_periods(self):
    self.X = self.X.apply(
                 lambda x: re.sub('[%s]' %
                  re.escape(self.remove_punctuations), '' , x))
    return self

  def remove_all_punctuations(self):
    self.X = self.X.apply(lambda x: re.sub('[%s]' %
                          re.escape(string.punctuation), '' , x))
    return self

  def remove_double_spaces(self):
    self.X = self.X.apply(lambda x: re.sub(' +', ' ', x))
    return self

  def fix_typos(self):
    self.X = self.X.apply(lambda x: str(TextBlob(x).correct()))
    return self

  def remove_stopwords(self):
    # remove stop words from token list in each column
    self.X = self.X.apply(
            lambda x: " ".join([ word for word in x.split() 
                     if word not in self.sw_nltk]) )
    return self

  def lemmatize(self):
    lemmatizer = WordNetLemmatizer()
    self.X = self.X.apply(lambda x: lemmatize_pos_tagged_text(
                           x, lemmatizer, self.pos_tag_dict))
    return self

  def get_processed_text(self):
    return self.X

In [10]:
globalization_data = data.loc[data['era'] == 'Globalization']
globalization_data.reset_index(drop=True, inplace=True)

In [11]:
SA_preproc = NltkPreprocessingSteps(globalization_data['transcript'])
SA_processed = \
    SA_preproc \
    .remove_html_tags()\
    .replace_diacritics()\
    .expand_contractions()\
    .remove_double_spaces()\
    .get_processed_text()

In [12]:
txt_preproc = NltkPreprocessingSteps(SA_processed)
TM_processed = \
    txt_preproc \
    .remove_numbers()\
    .remove_punctuations_except_periods()\
    .lemmatize()\
    .remove_all_punctuations()\
    .remove_stopwords()\
    .get_processed_text()

In [13]:
for i in range(len(TM_processed)):
    TM_processed[i] = list(TM_processed[i].split(' '))

In [14]:
from gensim.models import Phrases
bigram = Phrases(TM_processed, min_count=20)
for i in range(len(TM_processed)):
    for token in bigram[TM_processed[i]]:
        if '_' in token:
            TM_processed[i].append(token)

In [29]:
dictionary = Dictionary(TM_processed)
dictionary.filter_extremes(no_below=5, no_above=0.5)

corpus = [dictionary.doc2bow(text) for text in TM_processed]

2023-10-05 21:26:28,043 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-10-05 21:26:28,636 : INFO : built Dictionary<18315 unique tokens: ['abroad', 'absorb', 'accept', 'accord', 'across']...> from 216 documents (total 441819 corpus positions)
2023-10-05 21:26:28,637 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<18315 unique tokens: ['abroad', 'absorb', 'accept', 'accord', 'across']...> from 216 documents (total 441819 corpus positions)", 'datetime': '2023-10-05T21:26:28.637249', 'gensim': '4.3.0', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2023-10-05 21:26:28,667 : INFO : discarding 13361 tokens: [('absorb', 3), ('across', 151), ('act', 154), ('action', 138), ('againand', 1), ('ago', 166), ('allow', 139), ('also', 199), ('always', 159), ('america', 206)]...
2023-10-05 21:26:28,668 : INFO : keeping 4954 tokens which were in no less than 5 and no more tha

In [16]:
import logging
from gensim.models.coherencemodel import CoherenceModel
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [37]:
chunksize = 50
passes = 25
iterations = 200
eval_every = None
for i in range(3, 11):
    num_topics = i
    model = gensim.models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
    )
    topics = model.show_topics(
    num_topics=num_topics,
    num_words=10,
    log=False,
    formatted=False)
    cm = CoherenceModel(model=model, corpus=corpus, coherence='u_mass')
    coherence = cm.get_coherence()
    print(i)
    print(coherence)

2023-10-05 21:34:33,282 : INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
2023-10-05 21:34:33,284 : INFO : using serial LDA version on this node
2023-10-05 21:34:33,286 : INFO : running online (multi-pass) LDA training, 3 topics, 25 passes over the supplied corpus of 216 documents, updating model once every 50 documents, evaluating perplexity every 0 documents, iterating 200x with a convergence threshold of 0.001000
2023-10-05 21:34:33,286 : INFO : PROGRESS: pass 0, at document #50/216
2023-10-05 21:34:33,447 : INFO : optimized alpha [0.44415888, 0.52267575, 0.40417832]
2023-10-05 21:34:33,448 : INFO : merging changes from 50 documents into a model of 216 documents
2023-10-05 21:34:33,449 : INFO : topic #0 (0.444): 0.006*"tax" + 0.005*"health_care" + 0.004*"bill" + 0.004*"tonight" + 0.004*"cut" + 0.004*"governor" + 0.003*"mission" + 0.003*"budget" + 0.003*"somalia" + 0.003*"education"
2023-10-05 21:34:33,450 : INFO : topic #1 (0.523): 0.006*"tax" + 0.00

3
-0.9193012536334351


2023-10-05 21:34:40,450 : INFO : topic #0 (0.372): 0.008*"senator" + 0.007*"cut" + 0.007*"tax" + 0.005*"dole" + 0.005*"tonight" + 0.004*"clinton" + 0.004*"ought" + 0.004*"reform" + 0.004*"bill" + 0.004*"budget"
2023-10-05 21:34:40,451 : INFO : topic #1 (0.386): 0.006*"tax" + 0.005*"health_care" + 0.005*"cut" + 0.005*"tonight" + 0.004*"education" + 0.004*"budget" + 0.004*"bill" + 0.004*"drug" + 0.003*"reform" + 0.003*"lot"
2023-10-05 21:34:40,453 : INFO : topic #2 (0.327): 0.005*"iraq" + 0.005*"tonight" + 0.004*"kuwait" + 0.004*"soviet" + 0.004*"tax" + 0.003*"united_nation" + 0.003*"health_care" + 0.003*"aggression" + 0.003*"saddam" + 0.003*"weapon"
2023-10-05 21:34:40,453 : INFO : topic #3 (0.316): 0.006*"tax" + 0.005*"troop" + 0.004*"cut" + 0.004*"tonight" + 0.004*"agreement" + 0.004*"senator" + 0.003*"deficit" + 0.003*"health_care" + 0.003*"budget" + 0.003*"somalia"
2023-10-05 21:34:40,454 : INFO : topic diff=1.933885, rho=1.000000
2023-10-05 21:34:40,458 : INFO : PROGRESS: pass 0, a

4
-0.8753403239455273


2023-10-05 21:34:48,327 : INFO : merging changes from 50 documents into a model of 216 documents
2023-10-05 21:34:48,329 : INFO : topic #0 (0.266): 0.008*"tax" + 0.007*"cut" + 0.006*"senator" + 0.006*"budget" + 0.005*"bill" + 0.004*"welfare" + 0.004*"tonight" + 0.004*"drug" + 0.004*"deficit" + 0.004*"percent"
2023-10-05 21:34:48,330 : INFO : topic #1 (0.251): 0.007*"tonight" + 0.006*"tax" + 0.004*"education" + 0.004*"cut" + 0.004*"parent" + 0.003*"welfare" + 0.003*"governor" + 0.003*"drug" + 0.003*"health_care" + 0.003*"reform"
2023-10-05 21:34:48,331 : INFO : topic #2 (0.248): 0.006*"tax" + 0.004*"bush" + 0.004*"cut" + 0.004*"nuclear" + 0.003*"governor" + 0.003*"process" + 0.003*"europe" + 0.003*"international" + 0.003*"soviet" + 0.003*"tonight"
2023-10-05 21:34:48,332 : INFO : topic #3 (0.202): 0.012*"health_care" + 0.004*"tonight" + 0.004*"senator" + 0.004*"insurance" + 0.004*"bill" + 0.003*"budget" + 0.003*"reform" + 0.003*"drug" + 0.003*"doctor" + 0.003*"tax"
2023-10-05 21:34:48,3

...

8
-1.5354474913631133


2023-10-05 21:35:23,366 : INFO : optimized alpha [0.09988244, 0.16645145, 0.116575085, 0.15147118, 0.11091021, 0.15290207, 0.14353955, 0.08489463, 0.10555194]
2023-10-05 21:35:23,369 : INFO : merging changes from 50 documents into a model of 216 documents
2023-10-05 21:35:23,371 : INFO : topic #7 (0.085): 0.012*"soviet" + 0.008*"iraq" + 0.006*"europe" + 0.006*"kuwait" + 0.006*"soviet_union" + 0.005*"bosnia" + 0.005*"saddam" + 0.004*"coalition" + 0.004*"weapon" + 0.004*"troop"
2023-10-05 21:35:23,372 : INFO : topic #0 (0.100): 0.008*"tax" + 0.008*"senator" + 0.007*"dole" + 0.006*"cut" + 0.006*"clinton" + 0.005*"bush" + 0.005*"tonight" + 0.005*"bill" + 0.004*"money" + 0.004*"lot"
2023-10-05 21:35:23,372 : INFO : topic #3 (0.151): 0.009*"tax" + 0.008*"cut" + 0.006*"senator" + 0.005*"dole" + 0.005*"tonight" + 0.005*"bill" + 0.005*"budget" + 0.005*"deficit" + 0.004*"education" + 0.004*"lot"
2023-10-05 21:35:23,373 : INFO : topic #5 (0.153): 0.007*"tax" + 0.006*"tonight" + 0.005*"cut" + 0.00

9
-2.2884467293666315


2023-10-05 21:35:33,641 : INFO : optimized alpha [0.13122664, 0.13026246, 0.08620009, 0.13165818, 0.12515992, 0.10552323, 0.1352577, 0.092780784, 0.13135016, 0.12071874]
2023-10-05 21:35:33,644 : INFO : merging changes from 50 documents into a model of 216 documents
2023-10-05 21:35:33,647 : INFO : topic #2 (0.086): 0.009*"governor" + 0.009*"bush" + 0.008*"troop" + 0.008*"clinton" + 0.007*"tax" + 0.006*"somalia" + 0.006*"minute" + 0.005*"cut" + 0.005*"agreement" + 0.005*"tonight"
2023-10-05 21:35:33,648 : INFO : topic #7 (0.093): 0.008*"process" + 0.008*"cooperation" + 0.006*"gorbachev" + 0.006*"soviet" + 0.005*"nuclear" + 0.005*"europe" + 0.005*"cut" + 0.005*"international" + 0.004*"president_gorbachev" + 0.004*"area"
2023-10-05 21:35:33,649 : INFO : topic #8 (0.131): 0.007*"cut" + 0.006*"tax" + 0.006*"education" + 0.005*"tonight" + 0.005*"budget" + 0.004*"parent" + 0.004*"welfare" + 0.004*"college" + 0.004*"bill" + 0.004*"reform"
2023-10-05 21:35:33,650 : INFO : topic #3 (0.132): 0.0

10
-1.2425765516639946


In [34]:
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=model, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()
coherence

-3.112035278488255

In [ ]:
class Topic:
    def __init__(self, topic_list):
        self.topics = topic_list
        self.neg = 0
        self.neu = 0
        self.pos = 0
        self.sent_compound = 0
        self.sent_avg = 0
    def sentiment(self, sentiment, score):
        if sentiment == 'neg':
            self.neg += 1
        elif sentiment == 'pos':
            self.pos += 1
        elif sentiment == 'neu':
            self.neu += 1
        self.sent_compound += score
        self.sent_avg = self.sent_compound/(self.neg + self.pos + self.neu)

In [35]:
for topic_id, topic in topics:
    print([word for word, _ in topic])

['unlikely', 'posturing', 'alcohol', 'immune', 'agreement', 'taliban', 'afghan', 'deficit_reduction', 'virus', 'putin']
['vein', 'virus', 'van', 'minnesota', 'coronavirus', 'ventilator', 'goodpaying', 'jesse', 'generational', 'multiple']
['achievable', 'vitality', 'bend', 'available', 'assessment', 'amnesty', 'amazing', 'adequate', 'complexity', 'widely']
['achievable', 'vitality', 'bend', 'available', 'assessment', 'amnesty', 'amazing', 'adequate', 'complexity', 'widely']
['achievable', 'vitality', 'bend', 'available', 'assessment', 'amnesty', 'amazing', 'adequate', 'complexity', 'widely']
['decision', 'al', 'yous', 'process', 'nuclear', 'israel', 'within', 'international', 'iran', 'reason']
['lot', 'really', 'governor', 'applause', 'case', 'deal', 'incredible', 'actually', 'want_thank', 'thank_much']
['senator', 'clinton', 'dole', 'welfare', 'ought', 'budget', 'cuban', 'education', 'cuba', 'president_clinton']
['election', 'please', 'test', 'china', 'saw', 'supply', 'win', 'tremendou

In [ ]:
topics_list = []
for topic_id, topic in topics:
    topics_list.append(Topic([word for word, _ in topic]))

In [ ]:
for i in range(len(SA_processed)):
    SA_processed[i] = nltk.sent_tokenize(SA_processed[i])

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
globalization_data['vader'] =pd.Series(dtype='object')

In [ ]:
vader_model = SentimentIntensityAnalyzer()
for i in range(len(SA_processed)):
    score_list = []
    for j in range(len(SA_processed[i])):
        score_list.append(vader_model.polarity_scores(SA_processed[i][j])['compound'])
    globalization_data['vader'][i] = score_list

In [ ]:
globalization_data['sentiment'] =pd.Series(dtype='object')

In [ ]:
for i in range(len(globalization_data)):
    sentiment_list = []
    for j in range(len(globalization_data['vader'][i])):
        if globalization_data['vader'][i][j] > 0.05:
            sentiment_list.append('pos')
        elif -0.05 <= globalization_data['vader'][i][j] <= 0.05:
            sentiment_list.append('neu')
        elif globalization_data['vader'][i][j] < 0.05:
            sentiment_list.append('neg')
    globalization_data['sentiment'][i] = sentiment_list

In [ ]:
def common_member(a, b):
    a_set = set(a)
    b_set = set(b)
    if len(a_set.intersection(b_set)) > 0:
        return True 
    return False

In [ ]:

for i in range(len(SA_processed)):
    txt_preproc = NltkPreprocessingSteps(pd.Series(SA_processed[i]))
    post_lemmatize = \
        txt_preproc \
        .remove_numbers()\
        .remove_punctuations_except_periods()\
        .lemmatize()\
        .remove_all_punctuations()\
        .remove_stopwords()\
        .get_processed_text()
    for l in range(len(post_lemmatize)):
        post_lemmatize[l] = post_lemmatize[l].split(' ')
    bigram = Phrases(post_lemmatize, min_count=20)
    for j in range(len(post_lemmatize)):
        for token in bigram[post_lemmatize[j]]:
            if '_' in token:
                post_lemmatize[j].append(token)
    for k in range(len(post_lemmatize)):
        for topic in topics_list:
            if common_member(topic.topics, post_lemmatize[k]):
                topic.sentiment(globalization_data['sentiment'][i][k], globalization_data['vader'][i][k])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
topic_names = ("1", "2", "3", '4', '5', '6', '7', '8', '9', '10')

sentiment = {
    'Negative': (topics_list[0].neg, topics_list[1].neg, topics_list[2].neg, topics_list[3].neg, topics_list[4].neg, topics_list[5].neg, topics_list[6].neg, topics_list[7].neg, topics_list[8].neg, topics_list[9].neg),
    'Neutral': (topics_list[0].neu, topics_list[1].neu, topics_list[2].neu, topics_list[3].neu, topics_list[4].neu, topics_list[5].neu, topics_list[6].neu, topics_list[7].neu, topics_list[8].neu, topics_list[9].neu),
    'Positive': (topics_list[0].pos, topics_list[1].pos, topics_list[2].pos, topics_list[3].pos, topics_list[4].pos, topics_list[5].pos, topics_list[6].pos, topics_list[7].pos, topics_list[8].pos, topics_list[9].pos)
}

x = np.arange(len(topic_names))
width = 0.25  
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

fig.set_size_inches(18.5, 10.5)
for attribute, measurement in sentiment.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Number of matches to each topic')
ax.set_title('Topic ')
ax.set_xticks(x + width, topic_names)
ax.legend(loc='upper left', ncols=3)
fig.savefig('test2png.png', dpi=100)
plt.show